<a href="https://colab.research.google.com/github/tkn13/datasci_project/blob/dev%2Fdataprep/Prae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install skimpy

import pandas as pd
from skimpy import skim
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter_client 7.4.9
    Uninstalling jupyter_client-7.4.9:
      Successfully uninstalled jupyter_client-7.4.9
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 6.17.1
    Uninstalling ipykernel-6

In [ ]:
df = pd.read_csv("clean_data03.csv")
df.head()

In [ ]:
skim(df)

In [ ]:
df["Has_lapsed"] = df["Date_lapse"].notnull().astype(int)

In [ ]:
df = df.dropna(subset=[
    "Type_risk","Area","Second_driver","Length_of_vehicle_usage","Year_matriculation",
    "Power","Cylinder_capacity","Value_vehicle","N_doors","Weight"
])

In [ ]:
df["Type_fuel"] = df["Type_fuel"].fillna("Unknown")

In [ ]:
fuel_map = {
    "P": 1,        # เบนซิน
    "D": 2,        # ดีเซล
    "Unknown": 0    # ไม่ทราบ
}

df["Type_fuel"] = df["Type_fuel"].map(fuel_map).astype(int)

In [ ]:
df = df.sort_values(["ID", "Date_last_renewal"])

df["Years_with_company"] = (
    df.groupby("ID")
      .cumcount() + 1
)

In [ ]:
df.boxplot(column="Length", by="Type_risk", figsize=(8,5))
plt.title("Length by Type_risk")
plt.suptitle("")
plt.show()

In [ ]:
df["Length"] = (
    df.groupby("Type_risk")["Length"]
      .transform(lambda x: x.fillna(x.mean()))
)

In [ ]:
df["Type_risk"].value_counts().sort_index().rename_axis("Type_risk").reset_index(name="count")

In [ ]:
df_risk3 = df[df["Type_risk"] == 3].copy()

In [ ]:
df_risk3["Type_risk"].value_counts().sort_index().rename_axis("Type_risk").reset_index(name="count")

In [ ]:
(df_risk3["Power"] == 0).sum()

In [ ]:
df_risk3["Type_fuel"].value_counts().sort_index().rename_axis("Type_fuel").reset_index(name="count")

In [ ]:
df_risk3.drop(['Type_risk', 'Date_start_contract', 'Date_last_renewal', 'Date_next_renewal', 'Date_birth', 'Date_driving_licence', 'Seniority', 'Date_lapse', 'Year_matriculation'], axis=1, inplace=True)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

df_risk3.to_csv("/content/drive/MyDrive/clean_data06_OnlyRisk3.csv", index=False)